In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 10,
                                   zoom_range = 0.25,
                                   horizontal_flip = True,
                                   vertical_flip=True,
                                   width_shift_range = 0.2,
                                   rotation_range=20,
                                   fill_mode = 'nearest')
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/ML_miniproject/archive (1)/chest_xray (1)/train (1)',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

test_datagen = ImageDataGenerator(rescale = 1/255)
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/ML_miniproject/archive (1)/chest_xray (1)/test (1)',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'binary')

val_datagen = ImageDataGenerator(rescale = 1/255)
val_set = test_datagen.flow_from_directory('/content/drive/MyDrive/ML_miniproject/archive (1)/chest_xray (1)/val (1)',
                                            target_size=(224,224),
                                           batch_size = 2,
                                           class_mode = 'binary')

Found 5219 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                  patience=10)

In [ ]:
lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                          patience=8)

In [ ]:
def create_model():
  resnet_model = tf.keras.applications.ResNet50V2(
        weights='imagenet',
        include_top = False,
        input_shape = (224,224,3)
    )
  for layer in resnet_model.layers:
        layer.trainable=False
  x = resnet_model.output
  x=(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))(x)
  x=(tf.keras.layers.Flatten())(x)
  x=(tf.keras.layers.Dense(units=128, activation='relu'))(x)
  predictions=(tf.keras.layers.Dense(units=1, activation='sigmoid'))(x)
  res_model = tf.keras.Model(inputs=resnet_model.input, outputs=predictions)
  res_model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
  return res_model

In [ ]:
res_model_new = create_model()

94683136/94668760 [==============================] - 1s 0us/step


In [ ]:
res_model_new.fit(x = training_set, validation_data=val_set,steps_per_epoch=100,callbacks=[early_stopping,lr], epochs = 7,batch_size=32)

Epoch 1/7
100/100 [==============================] - 691s 5s/step - loss: 0.5841 - accuracy: 0.8568 - val_loss: 0.3316 - val_accuracy: 0.9375 - lr: 0.0010
Epoch 2/7
100/100 [==============================] - 517s 5s/step - loss: 0.2000 - accuracy: 0.9205 - val_loss: 0.3385 - val_accuracy: 0.9375 - lr: 0.0010
Epoch 3/7
100/100 [==============================] - 526s 5s/step - loss: 0.1547 - accuracy: 0.9357 - val_loss: 0.3907 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 4/7
100/100 [==============================] - 524s 5s/step - loss: 0.1569 - accuracy: 0.9350 - val_loss: 0.5733 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 5/7
100/100 [==============================] - 517s 5s/step - loss: 0.1314 - accuracy: 0.9480 - val_loss: 0.4749 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 6/7
100/100 [==============================] - 526s 5s/step - loss: 0.1307 - accuracy: 0.9502 - val_loss: 0.3768 - val_accuracy: 0.9375 - lr: 0.0010
Epoch 7/7
100/100 [==============================] - 517s 5s/step - lo

In [ ]:
from keras.callbacks import ModelCheckpoint
mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True)
mc

In [ ]:
res_model_new.save('final_model_resnet.h5') 

In [ ]:
score = res_model_new.evaluate(training_set)

print("Train Loss: ", score[0])
print("Train Accuracy: ", score[1])

score = res_model_new.evaluate(test_set)
print("\nTest loss: ", score[0])
print("Test Accuracy: ", score[1])

164/164 [==============================] - 833s 5s/step - loss: 0.1612 - accuracy: 0.9373
Train Loss:  0.16122178733348846
Train Accuracy:  0.9373443126678467
20/20 [==============================] - 219s 11s/step - loss: 0.2674 - accuracy: 0.9087

Test loss:  0.2673688530921936
Test Accuracy:  0.9086538553237915
